In [1]:

import os, sys

os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 --repositories https://repos.spark-packages.org/ pyspark-shell '
spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
os.environ["PYSPARK_PYTHON"] = 'python3'
#spark = exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

    


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("lab05").getOrCreate() 

In [3]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
        

In [4]:

schema = StructType([ \
    StructField("id",IntegerType(),True), \
    StructField("some",StringType(),True), \
    StructField("Rating",IntegerType(),True), \
    StructField("Userid", IntegerType(), True)
  ])

In [5]:
df = spark.read.csv(
    'hdfs:/labs/lab05data/ml-100k/u.data',
    schema=schema, header=False, sep='\t')

In [6]:
result = {"hist_film":[],"hist_all":[]}


In [7]:
result['hist_film'] = [int(row['count']) for row in df.filter("some==234").groupby('Rating').count().sort('Rating').drop('Rating').collect()]

In [8]:
result['hist_all'] = [int(row['count']) for row in df.groupby('Rating').count().sort('Rating').drop('Rating').collect()]

In [9]:
result

{'hist_film': [8, 19, 69, 116, 68],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [10]:
import json
with open("lab05.json", "w") as outfile:
    json.dump(result, outfile)
